<a href="https://colab.research.google.com/github/Balavignesh527/aidsa/blob/main/City_demand_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving enhanced_tn_product_demand.csv to enhanced_tn_product_demand.csv


In [1]:
import pandas as pd

df = pd.read_csv("enhanced_tn_product_demand.csv")
df.head()


,City,Product,Average_Price,Monthly_Demand,Population,Average_Income,Urbanization_Level,Temperature,Rainfall,Festival_Season,Category,Shelf_Life_Days,Is_Staple_Item,Brand_Popularity_Score,Num_Competitors,Marketing_Spend,Stock_Availability,Price_to_Income_Ratio,City_Category
0,Chennai,Rice,46.08,28,1604384,10027,57.60929,20.253437,95.560114,0,Food,180,1,9,9,460407.310930,77.603714,0.004596,Tier-2
1,Chennai,Wheat,48.11,27,1604384,10027,57.60929,20.253437,95.560114,0,Food,180,1,8,33,755940.849392,72.227406,0.004798,Tier-2
2,Chennai,Sugar,56.53,23,1604384,10027,57.60929,20.253437,95.560114,0,Food,365,1,7,43,141145.949779,73.113241,0.005638,Tier-2
3,Chennai,Milk,53.01,24,1604384,10027,57.60929,20.253437,95.560114,0,Beverage,7,1,10,38,551446.358655,94.387852,0.005287,Tier-2
4,Chennai,Tea,119.08,8,1604384,10027,57.60929,20.253437,95.560114,0,Beverage,365,0,9,47,550785.124130,71.166211,0.011876,Tier-2


In [2]:
!pip install streamlit scikit-learn xgboost pandas numpy matplotlib seaborn


In [3]:
import streamlit as st
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
st.set_page_config(page_title="City-Based Product Demand Predictor", page_icon="📈")
st.title("📈 City-Based Product Demand Predictor")
df = pd.read_csv("enhanced_tn_product_demand.csv")
le_city = LabelEncoder()
le_product = LabelEncoder()
le_category = LabelEncoder()
le_city_cat = LabelEncoder()

df["City_encoded"] = le_city.fit_transform(df["City"])
df["Product_encoded"] = le_product.fit_transform(df["Product"])
df["Category_encoded"] = le_category.fit_transform(df["Category"])
df["City_Category_encoded"] = le_city_cat.fit_transform(df["City_Category"])
feature_cols = [
    "City_encoded", "Product_encoded", "Category_encoded", "City_Category_encoded",
    "Average_Price", "Population", "Average_Income", "Urbanization_Level",
    "Temperature", "Rainfall", "Festival_Season", "Shelf_Life_Days",
    "Is_Staple_Item", "Brand_Popularity_Score", "Num_Competitors",
    "Marketing_Spend", "Stock_Availability", "Price_to_Income_Ratio"
]

X = df[feature_cols]
y = df["Monthly_Demand"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)
joblib.dump(model, "enhanced_demand_model.pkl")
joblib.dump(le_city, "city_encoder.pkl")
joblib.dump(le_product, "product_encoder.pkl")
joblib.dump(le_category, "category_encoder.pkl")
joblib.dump(le_city_cat, "city_category_encoder.pkl")

st.success("✅ Enhanced model trained successfully!")
selected_city = st.selectbox("Select a City", sorted(df["City"].unique()))
city_data = df[df["City"] == selected_city].copy()
city_data["Predicted_Demand"] = model.predict(city_data[feature_cols])
top10 = city_data.nlargest(10, "Predicted_Demand")[["Product", "Predicted_Demand"]]
st.subheader(f"🏆 Top 10 Predicted High-Demand Products in {selected_city}")
st.dataframe(top10, use_container_width=True)
st.bar_chart(top10.set_index("Product")["Predicted_Demand"])


2025-11-11 03:29:19.684 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 03:29:19.689 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 03:29:20.786 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-11 03:29:20.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 03:29:20.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 03:29:36.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 03:29:36.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()

In [4]:
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
model = joblib.load("enhanced_demand_model.pkl")
le_city = joblib.load("city_encoder.pkl")
le_product = joblib.load("product_encoder.pkl")
le_category = joblib.load("category_encoder.pkl")
le_city_cat = joblib.load("city_category_encoder.pkl")

df = pd.read_csv("enhanced_tn_product_demand.csv")
feature_cols = [
    "City_encoded", "Product_encoded", "Category_encoded", "City_Category_encoded",
    "Average_Price", "Population", "Average_Income", "Urbanization_Level",
    "Temperature", "Rainfall", "Festival_Season", "Shelf_Life_Days",
    "Is_Staple_Item", "Brand_Popularity_Score", "Num_Competitors",
    "Marketing_Spend", "Stock_Availability", "Price_to_Income_Ratio"
]
def predict_demand(city):
    city_code = le_city.transform([city])[0]
    city_data = df[df["City"] == city].copy()
    city_data["City_encoded"] = city_code
    city_data["Product_encoded"] = le_product.transform(city_data["Product"])
    city_data["Category_encoded"] = le_category.transform(city_data["Category"])
    city_data["City_Category_encoded"] = le_city_cat.transform(city_data["City_Category"])
    city_data["Predicted_Demand"] = model.predict(city_data[feature_cols])
    top_products = (
        city_data[["Product", "Predicted_Demand"]]
        .sort_values(by="Predicted_Demand", ascending=False)
        .head(5)
    )

    return top_products
city_name = "Chennai"
top_products = predict_demand(city_name)

print(f"\n🏙️ Top 5 Predicted High-Demand Products in {city_name}:\n")
for _, row in top_products.iterrows():
    print(f"🔸 {row['Product']}: predicted demand = {round(row['Predicted_Demand'], 2)}")



🏙️ Top 5 Predicted High-Demand Products in Chennai:

🔸 SoftDrinks: predicted demand = 39.01
🔸 Biscuits: predicted demand = 31.99
🔸 Rice: predicted demand = 29.2
🔸 Wheat: predicted demand = 26.99
🔸 Vegetables: predicted demand = 25.43


In [5]:
!pip install streamlit pyngrok plotly joblib pandas --quiet
!pip install xlsxwriter



In [6]:
!pip install --upgrade streamlit

In [10]:
!streamlit run app.py --server.port 8501 &



2025-11-11 03:35:26.180 Port 8501 is already in use


In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import io
import plotly.express as px
import numpy as np
from sklearn.preprocessing import LabelEncoder

st.set_page_config(page_title="City-Based Product Demand Predictor", page_icon="📈", layout="wide")
if "theme" not in st.session_state:
    st.session_state.theme = "light"
if "price_multiplier" not in st.session_state:
    st.session_state.price_multiplier = 1.0
if "selected_row" not in st.session_state:
    st.session_state.selected_row = None

def toggle_theme():
    st.session_state.theme = "dark" if st.session_state.theme == "light" else "light"

st.markdown(
    """
    <style>
    :root { --card-bg: rgba(255,255,255,0.75); --muted:#6c757d; }
    .dark :root { --card-bg: rgba(20,20,20,0.6); --muted:#9aa3b2; }
    .header {display:flex;align-items:center;gap:12px}
    .title {font-size:26px;font-weight:700}
    .subtitle {color:var(--muted);margin-top:-6px}
    .metric-card{padding:16px;border-radius:12px;background:var(--card-bg);box-shadow:0 6px 18px rgba(0,0,0,0.06);border:1px solid rgba(0,0,0,0.04)}
    .chips {display:flex;flex-wrap:wrap;gap:8px}
    .chip {padding:6px 10px;border-radius:20px;background:#f1f3f5;border:1px solid rgba(0,0,0,0.06);cursor:pointer}
    .chip-selected {background:#0d6efd;color:white;border:1px solid rgba(0,0,0,0.1)}
    .footer {color:var(--muted);font-size:13px;padding:12px;margin-top:18px;border-top:1px solid rgba(0,0,0,0.06);text-align:center}
    </style>
    """,
    unsafe_allow_html=True,
)

with st.container():
    st.markdown('<div class="header">', unsafe_allow_html=True)
    st.image("https://img.icons8.com/fluency/48/000000/commercial.png", width=48)
    st.markdown('<div><div class="title">📈 City-Based Product Demand Predictor</div><div class="subtitle">Interactive single-page dashboard — enhanced UX</div></div>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

with st.sidebar:
    st.markdown("### ⚙️ Controls")
    if st.button("Toggle Theme"):
        toggle_theme()
    st.markdown("### Data & Model")
    with st.spinner("Loading model & data..."):
        try:
            model = joblib.load("enhanced_demand_model.pkl")
            le_city = joblib.load("city_encoder.pkl")
            le_product = joblib.load("product_encoder.pkl")
            le_category = joblib.load("category_encoder.pkl")
            le_city_cat = joblib.load("city_category_encoder.pkl")
        except Exception as e:
            st.error("Model or encoder files not found. Run `train.py` to generate them.")
            st.stop()
        try:
            df = pd.read_csv("enhanced_tn_product_demand.csv")
        except Exception:
            st.error("CSV data not found. Place `enhanced_tn_product_demand.csv` in the app folder.")
            st.stop()

    st.markdown("---")
    st.markdown("### Filters")
    cities = sorted(df["City"].unique())
    selected_city = st.selectbox("City", cities)
    all_categories = sorted(df["Category"].unique())
    selected_categories = st.multiselect("Categories", all_categories, default=all_categories)
    top_n = st.slider("Top N products", 5, 25, 10)
    st.markdown("---")
    st.markdown("### Simulation")
    price_multiplier = st.slider("Price multiplier (simulate price change)", 0.5, 1.5, 1.0, 0.05, key="price_multiplier")
    if st.button("Reset Filters"):
        st.experimental_rerun()
    st.markdown("---")
    st.markdown("### Export")
    st.markdown("Download Top-N results below after predictions complete.")

feature_cols = [
    "City_encoded", "Product_encoded", "Category_encoded", "City_Category_encoded",
    "Average_Price", "Population", "Average_Income", "Urbanization_Level",
    "Temperature", "Rainfall", "Festival_Season", "Shelf_Life_Days",
    "Is_Staple_Item", "Brand_Popularity_Score", "Num_Competitors",
    "Marketing_Spend", "Stock_Availability", "Price_to_Income_Ratio"
]

city_data = df[(df["City"] == selected_city) & (df["Category"].isin(selected_categories))].copy()
if city_data.empty:
    st.warning(f"No products for {selected_city} with selected categories.")
    st.stop()

def safe_transform(encoder, series):
    classes = getattr(encoder, "classes_", None)
    if classes is None:
        raise ValueError("Encoder has no classes_")
    mapping = {v: i for i, v in enumerate(classes)}
    return series.map(mapping).astype(float)

try:
    city_data["City_encoded"] = safe_transform(le_city, city_data["City"])
    city_data["Product_encoded"] = safe_transform(le_product, city_data["Product"])
    city_data["Category_encoded"] = safe_transform(le_category, city_data["Category"])
    city_data["City_Category_encoded"] = safe_transform(le_city_cat, city_data["City_Category"])
except Exception as e:
    st.error("Encoding error: dataset contains unseen categories not present in encoders.")
    st.stop()

city_data["Average_Price_sim"] = city_data["Average_Price"] * st.session_state.price_multiplier
city_data_for_pred = city_data.copy()
city_data_for_pred["Average_Price"] = city_data_for_pred["Average_Price_sim"]

try:
    preds = model.predict(city_data_for_pred[feature_cols])
except Exception:
    st.error("Model prediction failed. Check feature alignment.")
    st.stop()

city_data["Predicted_Demand"] = np.round(preds, 0).astype(int)
top_n_data = city_data.nlargest(top_n, "Predicted_Demand").reset_index(drop=True)

total_top = int(top_n_data["Predicted_Demand"].sum())
avg_top = float(top_n_data["Predicted_Demand"].mean())
top_product = top_n_data.iloc[0]["Product"]
top_product_val = int(top_n_data.iloc[0]["Predicted_Demand"])

col1, col2, col3, col4 = st.columns([1.2,1,1,1])

with col1:
    st.markdown(f'<div class="metric-card"><h4>🏆 Top Product</h4><h2 style="margin:0">{top_product}</h2><div style="color:#6c757d;margin-top:6px">{top_product_val:,} units</div></div>', unsafe_allow_html=True)
with col2:
    st.markdown(f'<div class="metric-card"><h4>📦 Total Top {top_n} Demand</h4><h2 style="margin:0">{total_top:,}</h2><div style="color:#6c757d;margin-top:6px">units</div></div>', unsafe_allow_html=True)
with col3:
    st.markdown(f'<div class="metric-card"><h4>📊 Avg. Demand</h4><h2 style="margin:0">{avg_top:,.0f}</h2><div style="color:#6c757d;margin-top:6px">units per product</div></div>', unsafe_allow_html=True)
with col4:
    st.markdown(f'<div class="metric-card"><h4>⚖️ Price Multiplier</h4><h2 style="margin:0">x {st.session_state.price_multiplier:.2f}</h2><div style="color:#6c757d;margin-top:6px">Simulated effect</div></div>', unsafe_allow_html=True)

st.markdown("")

chart_col, table_col = st.columns([1.4,1])

with chart_col:
    st.subheader(f"Top {top_n} Products — {selected_city}")
    fig = px.bar(
        top_n_data.sort_values("Predicted_Demand", ascending=True),
        x="Predicted_Demand",
        y="Product",
        orientation="h",
        text="Predicted_Demand",
        labels={"Predicted_Demand":"Predicted Demand (units)"},
        title=f"Top {top_n} Predicted Products in {selected_city}"
    )
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.update_layout(transition={'duration':600,'easing':'cubic-in-out'}, height=560, margin=dict(l=120))
    st.plotly_chart(fig, use_container_width=True)

with table_col:
    st.subheader("Top-N Table")
    if "table_idx" not in st.session_state:
        st.session_state.table_idx = None
    display_table = top_n_data[["Product","Category","Average_Price_sim","Predicted_Demand","Brand_Popularity_Score"]].copy()
    display_table = display_table.rename(columns={"Average_Price_sim":"Price (Sim)","Predicted_Demand":"Predicted Demand","Brand_Popularity_Score":"Brand Score"})
    st.dataframe(display_table.style.format({"Price (Sim)":"{:.2f}","Predicted Demand":"{:,}"}), use_container_width=True, height=560)
    st.markdown("")
    csv_buffer = io.StringIO()
    top_n_data.to_csv(csv_buffer, index=False)
    csv_bytes = csv_buffer.getvalue().encode()
    excel_buffer = io.BytesIO()
    with pd.ExcelWriter(excel_buffer, engine="xlsxwriter") as writer:
        top_n_data.to_excel(writer, index=False, sheet_name="TopN")
    excel_bytes = excel_buffer.getvalue()
    st.download_button("⬇️ Download CSV", csv_bytes, file_name=f"{selected_city}_top_{top_n}.csv", mime="text/csv")
    st.download_button("⬇️ Download Excel", excel_bytes, file_name=f"{selected_city}_top_{top_n}.xlsx", mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    st.markdown("### Product Details")
    idx = st.number_input("Enter row number from Top-N (1..{0}) to open details".format(len(top_n_data)), min_value=1, max_value=len(top_n_data), value=1, step=1)
    if st.button("Show Product Details"):
        st.session_state.selected_row = int(idx)-1
        with st.modal("Product Details"):
            row = top_n_data.iloc[st.session_state.selected_row]
            st.markdown(f"### {row['Product']}")
            st.markdown(f"**Category:** {row['Category']}  ")
            st.markdown(f"**Simulated Price:** ₹{row['Average_Price_sim']:.2f}  ")
            st.markdown(f"**Predicted Demand:** {int(row['Predicted_Demand']):,} units  ")
            st.markdown(f"**Shelf Life (days):** {row.get('Shelf_Life_Days', 'N/A')}  ")
            st.markdown(f"**Brand Popularity:** {row.get('Brand_Popularity_Score', 'N/A')}  ")
            st.markdown(f"**Number of Competitors:** {row.get('Num_Competitors', 'N/A')}  ")
            st.markdown("---")
            st.markdown("#### Quick Actions")
            info_csv = io.StringIO()
            pd.DataFrame([row]).to_csv(info_csv, index=False)
            st.download_button("Download this product (CSV)", info_csv.getvalue().encode(), file_name=f"{row['Product']}_details.csv", mime="text/csv")
            st.button("Close")

st.markdown("---")
with st.expander("View all predictions for this city (sortable)"):
    st.dataframe(city_data.sort_values("Predicted_Demand", ascending=False), use_container_width=True)

st.markdown(f'<div class="footer">Made with ❤️ using Streamlit + XGBoost | v2.3 — Data: {len(df)} rows | City: {selected_city}</div>', unsafe_allow_html=True)


Overwriting app.py


In [8]:
from pyngrok import ngrok
import time
import os

NGROK_TOKEN = "346ucx3SeiLow0dujvyTgOO7IKp_6YBGSDf42SvQiMt1FFakZ"

# Authenticate ngrok
!ngrok authtoken {NGROK_TOKEN}
ngrok.kill()
os.system("streamlit run app.py --server.port 8501 &")
time.sleep(5)  # Wait for Streamlit to start
public_url = ngrok.connect(8501)
print("✅ Streamlit app is live at:")
print(public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Streamlit app is live at:
NgrokTunnel: "https://unpowdered-unconservatively-dean.ngrok-free.dev" -> "http://localhost:8501"
